# 2022 NHL Playoff Pool

In [1]:
import pandas as pd
import numpy as np
import os

In [236]:
# Initialize dicts
picks = {}
entries = {}
points = {}

In [76]:
brackets = os.listdir('brackets')

In [190]:
def read_brackets(brackets):
    """Read playoff bracket entries and save results as dict"""
    for bracket in brackets:
        df = pd.read_excel("brackets/"+bracket, sheet_name='teams')
        entrant_name = df['entrant_name'].dropna().item()
        if entrant_name in picks.keys():
            entrant_name+=' 2' 
        picks[entrant_name] = {
            'rd1_selections' : df['rd1_selections'].dropna().to_list(),
            'rd2_selections' : df['rd2_selections'].dropna().to_list(),
            'rd3_selections' : df['rd3_selections'].dropna().to_list(),
            'rd4_selections' : df['rd4_selections'].dropna().to_list(),
            'tie_breaker' : df['tie_breaker'].dropna().to_list()
        }
    return picks

In [191]:
read_brackets(brackets)

{'Cameron Harris': {'rd1_selections': ['NSH',
   'EDM',
   'COL',
   'MIN',
   'FLA',
   'TBL',
   'CAR',
   'NYR'],
  'rd2_selections': ['EDM', 'MIN', 'FLA', 'CAR'],
  'rd3_selections': ['EDM', 'FLA'],
  'rd4_selections': ['FLA'],
  'tie_breaker': ['Pasternak']},
 'Cameron Harris 2': {'rd1_selections': ['CGY',
   'EDM',
   'COL',
   'MIN',
   'FLA',
   'TOR',
   'BOS',
   'NYR'],
  'rd2_selections': ['CGY', 'COL', 'FLA', 'BOS'],
  'rd3_selections': ['COL', 'BOS'],
  'rd4_selections': ['COL'],
  'tie_breaker': ['Pasternak']}}

In [379]:
# Attempt class method to make this cleaner
class bracket_entry:
    """Class to manage bracket entries and calculate points"""
    def __init__(self, bracket_name):
        self.bracket_name = bracket_name
        
    def read_brackets(self, bracket_name):
        """Read playoff bracket entries and save results as dict"""
        df = pd.read_excel("brackets/"+bracket_name, sheet_name='teams')
        self.entrant_name = df['entrant_name'].dropna().item()
        if self.entrant_name in picks.keys():
            self.entrant_name = self.entrant_name + ' 2'

        picks[self.entrant_name] = {
            'rd1_selections' : df['rd1_selections'].dropna().to_list(),
            'rd2_selections' : df['rd2_selections'].dropna().to_list(),
            'rd3_selections' : df['rd3_selections'].dropna().to_list(),
            'rd4_selections' : df['rd4_selections'].dropna().to_list(),
            'tie_breaker' : df['tie_breaker'].dropna().to_list()
        }
        return picks
    
    def calc_points(self, picks, results):
        """Calculate pool points based on latest playoff results"""
        # Create for loop to go through each round in result and append points accordingly
        all_points = {}
        points_by_round = {
            'rd1': 2,
            'rd2': 4,
            'rd3': 8,
            'rd4': 17
        }
        for rd in results.keys():
            round_picks = picks[rd+'_selections']
            round_result = results[rd]
            round_points = len(set(round_picks) & set(round_result))*points_by_round[rd]
            all_points[rd+'_points'] = round_points
        return all_points

In [380]:
# Initialize entries
for n, bracket in enumerate(brackets):
    entries[str(n+1)] = bracket_entry(bracket)

In [381]:
entries['1'].bracket_name

'bracket_2022_CameronTest1.xlsx'

In [382]:
entries['2'].bracket_name

'bracket_2022_CameronTest.xlsx'

In [383]:
# Extract picks
for entry_num, entry in entries.items():
    picks = bracket_entry.read_brackets(bracket_entry, entry.bracket_name)

In [384]:
picks.keys()

dict_keys(['Cameron Harris', 'Cameron Harris 2'])

In [385]:
picks

{'Cameron Harris': {'rd1_selections': ['NSH',
   'EDM',
   'COL',
   'MIN',
   'FLA',
   'TBL',
   'CAR',
   'NYR'],
  'rd2_selections': ['EDM', 'MIN', 'FLA', 'CAR'],
  'rd3_selections': ['EDM', 'FLA'],
  'rd4_selections': ['FLA'],
  'tie_breaker': ['Pasternak']},
 'Cameron Harris 2': {'rd1_selections': ['CGY',
   'EDM',
   'COL',
   'MIN',
   'FLA',
   'TOR',
   'BOS',
   'NYR'],
  'rd2_selections': ['CGY', 'COL', 'FLA', 'BOS'],
  'rd3_selections': ['COL', 'BOS'],
  'rd4_selections': ['COL'],
  'tie_breaker': ['Pasternak']}}

In [386]:
# Results by round
results = {
    'rd1': [
        'CGY',
        'EDM',
        'COL',
        'MIN',
        'FLA',
        'TOR',
        'CAR',
        'PIT'
    ],
    'rd2': [
        'EDM',
        'COL',
        'FLA',
        'PIT'
    ],
    'rd3': [
        'COL',
        'FLA'
    ],
    'rd4': [
        'FLA'
    ]
}

In [387]:
results.keys()

dict_keys(['rd1', 'rd2', 'rd3', 'rd4'])

In [388]:
# Calculate points based on latest results
points = {}
for k, v in picks.items():
    points[k] = bracket_entry.calc_points(bracket_entry, v, results)

## TO DO
Investigate why this only works when I assign output from class method to a new dict, thougth I should be able to save the picks/results as attributes of each instance (e.g. self.picks, self.results)

In [389]:
points

{'Cameron Harris': {'rd1_points': 10,
  'rd2_points': 8,
  'rd3_points': 8,
  'rd4_points': 17},
 'Cameron Harris 2': {'rd1_points': 12,
  'rd2_points': 8,
  'rd3_points': 8,
  'rd4_points': 0}}

In [390]:
points_df = pd.DataFrame(points).T
points_df['total'] = points_df.iloc[:, :].sum(axis=1)
points_df

,rd1_points,rd2_points,rd3_points,rd4_points,total
Cameron Harris,10,8,8,17,43
Cameron Harris 2,12,8,8,0,28
